#Trabalho de Avaliação e Desempenho

Grupo:<br>
Bruno Hryniewicz - 115150099 <br>
Eduardo da Silva Barbosa - 116150432 <br>
Miguel Bicudo - 116033119 <br>
Nickolas Gomes Pinto - 114148170 <br>
Rafael Pais Cardoso - 116140788 <br>
Tainá da Silva Lima - 116165607 <br>

Professor(a): Daniel Sadoc <br>
Período: 2019.2

##Questão 6 

#### Código-base para a simulação

In [0]:
import numpy as np
import scipy as sp
import scipy.stats
import math
import time
import random
import operator
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from decimal import *

class Eventos:
  def __init__(self, tipo, fimEvento, duracaoServico):
    self.tipo = tipo
    self.fimEvento = fimEvento #Do ponto de vista do lambda é a chegada na fila e do ponto de vista de mi é o tempo de sair do servidor.
    self.duracaoServico = duracaoServico

class Cliente:
  def __init__(self, tipo, tempoResidual, tempoChegada, tempoEntradaServidor = -1, tempoSaida = -1  ):
    self.tempoResidual = tempoResidual
    self.tempoChegada = tempoChegada
    self.tempoEntradaServidor = tempoEntradaServidor
    self.tempoSaida = tempoSaida
    self.tipo = tipo
  def tempoNaFila (self):
    return self.tempoEntradaServidor - self.tempoChegada

  def tempoNoSistema (self):
    return self.tempoSaida - self.tempoChegada



#Simulador  
def simulaMG1b (λ1, λ2, µ1, µ2, nEventos, X = random.expovariate,  premp = False, tempoTotalSimulacao = 4000): # Simula uma fila M/G/1, retornando o E(N)
  linhaDoTempo = 0
  
  area1 = 0
  N01 = 0
  T01 = 0
  area2 = 0
  N02 = 0
  T02 = 0
  numeroTotalClienteT1 = 0
  numeroTotalClienteT2 = 0
  numeroTotalCliente = 0
  tempTotalServicoT1 = 0
  tempTotalServicoT2 = 0
  tempTotalServico = 0
  clientesServidos = []
  fila1 = []  #Fila1. OBS: A primeira posição NÃO representa o servidor.
  fila2 = []  #Fila2. OBS: A primeira posição NÃO representa o servidor.
  tempoResidualF1 = []
  tempoResidualF2 = []
  servidor = 0 #0 representa não ter ninguém no servidor; 1 cliente do tipo 1 e 2 cliente do tipo 2
  eventos = [] #Fila de eventos
  atualEvento = 0
  saida = 0
  xr1 = []
  xr2 = []
  xr = [] 
  U = []
  U_aux = []
  div = 0
  Gs = []
  primeira_entrada = 0
  Ntotal = []


  #Primeira ocorrência de eventos (λ1 e λ2)
  temp = random.expovariate(λ1)
  eventos.append(Eventos("Chegada1",temp , temp))
  if(λ2 != 0):
    temp = random.expovariate(λ2)
    eventos.append(Eventos("Chegada2", temp, temp))
  

  while(linhaDoTempo <= tempoTotalSimulacao):

    eventos.sort(key=lambda eventos: eventos.fimEvento)
    if (servidor != 0):
      Ntotal.append(len(fila1)+len(fila2)+1)
    else:
      Ntotal.append(len(fila1)+len(fila2))

    atualEvento = eventos[0] #Pega o primeiro evento
    eventos = eventos[1:] #Tira o primeiro evento da fila de eventos
    if (servidor == 0 ):
      Gs.append(linhaDoTempo - primeira_entrada)
      primeira_entrada = atualEvento.fimEvento
    linhaDoTempo = atualEvento.fimEvento #Avança a linha do tempo

    if(atualEvento.tipo == "Chegada1"):
      if (saida - linhaDoTempo > 0):
        U.append((len(fila1) * (1/µ1) + len(fila2) * (1/µ2) + saida - linhaDoTempo)  )
      else:
        U.append(0)
      if (servidor != 0):
        item_xr = saida - linhaDoTempo
        xr.append(item_xr)
        if (servidor == 1 ):
          xr1.append(item_xr)
        else:
          xr2.append(item_xr)
      fila1.append(Cliente(1,-1, linhaDoTempo))
      temp = random.expovariate(λ1)
      eventos.append(Eventos("Chegada1", linhaDoTempo + temp, temp)) #Evento de chegada do próximo cliente do tipo 1



      if(len(fila1) == 1 and servidor == 0):
        ClienteNoServidor = fila1[0]
        ClienteNoServidor.tempoEntradaServidor = linhaDoTempo
        clientesServidos.append(ClienteNoServidor)
        fila1 = fila1[1:]
        servidor = 1 #Cliente do tipo 1 sendo servido
        temp =  X(µ1)
        eventos.append(Eventos("Servico1", linhaDoTempo + temp, temp))
        saida = linhaDoTempo + temp
        tempoResidualF1.append(0)
        ClienteNoServidor.tempoSaida = saida
      
 
    elif(atualEvento.tipo == "Chegada2"):
      if (saida - linhaDoTempo > 0):
        U.append((len(fila1) * (1/µ1) + len(fila2) * (1/µ2) + saida - linhaDoTempo)  )
      else:
        U.append(0)
      if (servidor != 0):
        item_xr = saida - linhaDoTempo
        xr.append(item_xr)
        if (servidor == 1 ):
          xr1.append(item_xr)
        else:
          xr2.append(item_xr)
      fila2.append(Cliente(2, -1, linhaDoTempo))
      temp = random.expovariate(λ2)
      eventos.append(Eventos("Chegada2", linhaDoTempo +temp, temp)) #Evento de chegada do próximo cliente do tipo 2



      if(len(fila2) == 1 and servidor == 0 and len(fila1) == 0):
        ClienteNoServidor = fila2[0]
        ClienteNoServidor.tempoEntradaServidor = linhaDoTempo
        clientesServidos.append(ClienteNoServidor)
        fila2 = fila2[1:]
        servidor = 2 #Cliente do tipo 2 sendo servido
        temp =  X(µ2)
        eventos.append(Eventos("Servico2", linhaDoTempo + temp, temp))
        saida = linhaDoTempo + temp
        ClienteNoServidor.tempoSaida = saida


    elif(atualEvento.tipo == "Servico1" or atualEvento.tipo == "Servico2"):
      numeroTotalCliente += 1
      tempTotalServico += atualEvento.duracaoServico

      if(atualEvento.tipo == "Servico1"):
        tempTotalServicoT1 += atualEvento.duracaoServico
      elif(atualEvento.tipo == "Servico2"):
        tempTotalServicoT2 += atualEvento.duracaoServico

      if(servidor == 1):
        numeroTotalClienteT1 += 1
      elif(servidor == 2):  
        numeroTotalClienteT2 += 1

      servidor = 0

      if(len(fila1) > 0):
        ClienteNoServidor = fila1[0]
        ClienteNoServidor.tempoEntradaServidor = linhaDoTempo
        clientesServidos.append(ClienteNoServidor)
        fila1 = fila1[1:]
        servidor = 1
        temp =  X(µ1)
        eventos.append(Eventos("Servico1", linhaDoTempo + temp, temp))
        saida = linhaDoTempo + temp
        ClienteNoServidor.tempoSaida = saida
      elif(len(fila2) > 0):
        temp =  X(µ2)
        eventos.append(Eventos("Servico2", linhaDoTempo + temp, temp))
        ClienteNoServidor = fila2[0]
        ClienteNoServidor.tempoEntradaServidor = linhaDoTempo
        clientesServidos.append(ClienteNoServidor)
        fila2 = fila2[1:]
        servidor = 2
        saida = linhaDoTempo + temp
        ClienteNoServidor.tempoSaida = saida
 
    if(len(fila1) != N01):
      if(N01 > 0 ):
        area1 = area1 + (N01-1)*(linhaDoTempo - T01)

      N01 = len(fila1)
      T01 = linhaDoTempo

    if(len(fila2) != N02):
      if(N02 > 0 ):
        area2 = area2 + (N02-1)*(linhaDoTempo - T02)

      N02 = len(fila2)
      T02 = linhaDoTempo
  

  
  return Ntotal, Gs, U, area1/linhaDoTempo, area2/linhaDoTempo, (tempTotalServico/numeroTotalClienteT1), (tempTotalServico/numeroTotalClienteT2), (tempTotalServico/(numeroTotalClienteT1 + numeroTotalClienteT2 )),  numeroTotalCliente, (tempTotalServico/linhaDoTempo), (tempTotalServicoT1/linhaDoTempo), (tempTotalServicoT2/linhaDoTempo), clientesServidos, xr1, xr2, xr   #Retorna [Nq1],[Nq2], [X1], [X2], Número total de clientes servidos, ρ, ρ1, ρ2

#Chama o simulador diversas vezes e gera a média dos resultados
def simulaGeralb(λ1, λ2, µ1, µ2, nSimulacoes, X = random.expovariate):
  # Array para o resultado de cada uma das simulações
  Nq1_barras = []
  Nq2_barras = []
  X1_barras = []
  X2_barras = []
  X_barras = []
  T1_barras = []
  T2_barras = []
  W1_barras = []
  W2_barras = []
  W_barras = []
  Xr1_barras = []
  Xr2_barras = []
  Xr_barras = []
  TotaisClienteServidos = [] 
  ρ_s = []
  ρ1_s = []
  ρ2_s = []
  U_barras = []
  G_barras = []
  N_barras = []
  T = []
  

  # Médias das simulações
  media_simus_Nq = 0
  media_simus_W = 0
  media_simus_X = 0
  media_simus_N = 0
  media_simus_ρ = 0
  media_simus_T = 0
  media_simus_U = 0
  
  desvio_simus_Nq = 0
  desvio_simus_W = 0
  desvio_simus_X = 0
  desvio_simus_N = 0
  desvio_simus_ρ = 0
  desvio_simus_T = 0
  desvio_simus_U = 0

  for i in range(nSimulacoes): #Realiza n Simulacões.
    Ntotal,Gs, U, NQ1_barra, NQ2_barra, X1_barra, X2_barra, X_barra, totalClienteServidos, ρ, ρ1, ρ2, clientesServidos, xr1, xr2, xr = simulaMG1b(λ1, λ2, µ1, µ2, 4000, X) #Simulações com 1000 eventos cada
    
    w1_aux =  []
    w2_aux = []
    w_aux = []
    t = []
    for c in clientesServidos:
      if (c.tipo == 1):
        w1_aux.append(c.tempoNaFila())
      else:
        w2_aux.append(c.tempoNaFila())
      w_aux.append(c.tempoNaFila())
      t.append(c.tempoNoSistema())
    T.append(np.array(t).mean())
    Nq1_barras.append(NQ1_barra)
    Nq2_barras.append(NQ2_barra) 
    TotaisClienteServidos.append(totalClienteServidos)
    X1_barras.append(X1_barra)
    X2_barras.append(X2_barra)
    X_barras.append(X_barra)
    W1_barras.append(np.array(w1_aux).mean())
    W2_barras.append(np.array(w2_aux).mean())
    W_barras.append(np.array(w_aux).mean())
    ρ_s.append(ρ)
    ρ1_s.append(ρ1)
    ρ2_s.append(ρ2)
    T1_barras.append(W1_barras[i] + X1_barras[i])
    T2_barras.append(W1_barras[i] + X1_barras[i])
    Xr_barras.append(np.array(xr).mean())
    Xr1_barras.append(np.array(xr1).mean())
    Xr2_barras.append(np.array(xr2).mean())
    U_barras.append(np.array(U).mean())
    G_barras.append(np.array(Gs).mean())
    N_barras.append(np.array(Ntotal).mean())

  #Média para U_barra(Trabalho pendente) de todas as simulações
  media_simus_U = np.array(U_barras).mean()
  desvio_simus_U = np.std(np.array(U_barras))
  intervaloConf_U = []
  intervaloConf_U.append(media_simus_U - 1.96*(desvio_simus_U/math.sqrt(len(U_barras))))
  intervaloConf_U.append(media_simus_U + 1.96*(desvio_simus_U/math.sqrt(len(U_barras))))

  return T, N_barras, G_barras, U_barras, Nq1_barras, Nq2_barras, X1_barras, X2_barras, X_barras,  TotaisClienteServidos, ρ_s, ρ1_s, ρ2_s, W1_barras, W2_barras, W_barras, Xr1_barras, Xr2_barras, Xr_barras



In [0]:
l1 = 0.4
l2 = 0.2
u1 = 1
u2 = 0.5

T, N_barras, G_barras, U_barras, Nq1_barras, Nq2_barras, X1_barras, X2_barras, X_barras,  TotaisClienteServidos, ρ_s, ρ1_s, ρ2_s, W1_barras, W2_barras, W_barras, Xr1_barras, Xr2_barras, Xr_barras = simulaGeralb(l1, l2 , u1, u2, 1000, X = random.expovariate)


#### Item 1

In [0]:
def item1():
  print("E[W] = p1E[W1] + p2E[W2] ")
  media_W_barra = np.array(W_barras).mean()
  desvio_W_barra = np.std(np.array(W_barras))
  print("E[W] = ", media_W_barra, " +- ", 1.96*(desvio_W_barra/math.sqrt(len(W_barras)) ))
  media_W1_barra = np.array(W1_barras).mean()
  desvio_W1_barra = np.std(np.array(W1_barras))
  media_W2_barra = np.array(W2_barras).mean()
  desvio_W2_barra = np.std(np.array(W2_barras))
  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_ρ2_s = np.array(ρ2_s).mean()
  desvio_ρ2_s = np.std(np.array(ρ2_s))

  intconf_max = (media_W1_barra + 1.96*(desvio_W1_barra/math.sqrt(len(W1_barras)))) * (media_ρ1_s +  1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)))) + (media_W2_barra + 1.96*(desvio_W2_barra/math.sqrt(len(W2_barras)))) *( media_ρ2_s + 1.96*(desvio_ρ2_s/math.sqrt(len(ρ2_s)))) 
  intconf = intconf_max - (media_W1_barra *  media_ρ1_s + media_W2_barra *  media_ρ2_s)
  print ("p1E[W1] + p2E[W2] = ",  media_W1_barra *  media_ρ1_s + media_W2_barra *  media_ρ2_s, " +- ", intconf  )
  

item1()

E[W] = p1E[W1] + p2E[W2] 
E[W] =  4.5717627439425215  +-  0.060341607787303124
p1E[W1] + p2E[W2] =  4.673689741703605  +-  0.0840723637705354


#### Item 2

In [0]:
def item2():
  print("E[W1] = ρ1E[X1r]/(1 − ρ1) ")

  media_W1_barra = np.array(W1_barras).mean()
  desvio_W1_barra = np.std(np.array(W1_barras))
  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_Xr1_barra = np.array(Xr1_barras).mean()
  desvio_Xr1_barra = np.std(np.array(Xr1_barras))

  intconf_max = (( media_ρ1_s +  1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)) ))*  (media_Xr1_barra + 1.96*(desvio_Xr1_barra/math.sqrt(len(Xr1_barras)) ))) /(1 - ( media_ρ1_s +  1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)) )) )

  result = (media_ρ1_s *  media_Xr1_barra) /(1 - media_ρ1_s ) 
  intconf = intconf_max - result

  print("E[W1] = ", media_W1_barra, " +- ", 1.96*(desvio_W1_barra/math.sqrt(len(W1_barras)) ))
  print ("ρ1E[X1r]/(1 − ρ1) = ",  result, " +- ", intconf  )
  

item2()

E[W1] = ρ1E[X1r]/(1 − ρ1) 
E[W1] =  1.9943608989705208  +-  0.010818169654673615
ρ1E[X1r]/(1 − ρ1) =  0.6644880737330358  +-  0.004295808876271012


#### Item 3

In [0]:
def item3():
  print("E[W] = ρE[Xr]/(1 − ρ) ")

  media_W_barra = np.array(W_barras).mean()
  desvio_W_barra = np.std(np.array(W_barras))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))
  media_Xr_barra = np.array(Xr_barras).mean()
  desvio_Xr_barra = np.std(np.array(Xr_barras))

  intconf_max = (( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) ))*  (media_Xr_barra + 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)) ))) /(1 - ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) )) )

  result = (media_ρ_s *  media_Xr_barra) /(1 - media_ρ_s ) 
  intconf = intconf_max - result

  print("E[W1] = ", media_W_barra, " +- ", 1.96*(desvio_W_barra/math.sqrt(len(W_barras)) ))
  print ("ρE[Xr]/(1 − ρ) = ",  result, " +- ", intconf  )
  

item3()

E[W] = ρE[Xr]/(1 − ρ) 
E[W1] =  4.574447550269498  +-  0.05791517058992681
ρE[Xr]/(1 − ρ) =  5.9496313232841  +-  0.0715749858270609


#### Item 4

In [0]:
def item4():
  print("E[U] = ρE[Xr]/(1 − ρ) ")

  media_U_barra = np.array(U_barras).mean()
  desvio_U_barra = np.std(np.array(U_barras))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))
  media_Xr_barra = np.array(Xr_barras).mean()
  desvio_Xr_barra = np.std(np.array(Xr_barras))

  intconf_max = (( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) ))*  (media_Xr_barra + 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)) ))) /(1 - ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) )) )

  result = (media_ρ_s *  media_Xr_barra) /(1 - media_ρ_s ) 
  intconf = intconf_max - result

  print("E[U] = ", media_U_barra, " +- ", 1.96*(desvio_U_barra/math.sqrt(len(U_barras)) ))
  print ("ρE[Xr]/(1 − ρ) = ",  result, " +- ", intconf  )


item4()

E[U] = ρE[Xr]/(1 − ρ) 
E[U] =  6.019475061097268  +-  0.08311620190259747
ρE[Xr]/(1 − ρ) =  5.995562276556859  +-  0.07543966879652952


#### Item 5

In [0]:
def item5():
  print("E[X] = (ρ1/ρ)E[X1] + (ρ2/ρ)E[X2]")
  media_X_barra = np.array(X_barras).mean()
  desvio_X_barra = np.std(np.array(X_barras))
  print("E[X] = ", media_X_barra, " +- ", 1.96*(desvio_X_barra/math.sqrt(len(X_barras)) ))

  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_ρ2_s = np.array(ρ2_s).mean()
  desvio_ρ2_s = np.std(np.array(ρ2_s))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))


  intconf_max = (1/u1) * ((media_ρ1_s +  1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)))) / (media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))))  + (1/u2) *( (media_ρ2_s + 1.96*(desvio_ρ2_s/math.sqrt(len(ρ2_s)))) /(media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))) )
  result = ((1/u1)) *  (media_ρ1_s/ media_ρ_s) + ((1/u2) *  (media_ρ2_s/media_ρ_s))
  intconf = intconf_max - result
  print ("p1/p E[X1] + p2/p E[X2] = ",  result, " +- ", intconf  )

item5()

E[X] = (ρ1/ρ)E[X1] + (ρ2/ρ)E[X2]
E[X] =  1.3325144150605115  +-  0.0018927642793335388
p1/p E[X1] + p2/p E[X2] =  1.4988487932754395  +-  0.0012431882794490257


#### Item 6

In [0]:
def item6():
  print("E[Xr] = (ρ1/ρ)E[Xr1] + (ρ2/ρ)E[Xr2]")
  media_Xr_barra = np.array(Xr_barras).mean()
  desvio_Xr_barra = np.std(np.array(Xr_barras))
  print("E[Xr] = ", media_Xr_barra, " +- ", 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)) ))
  media_Xr1_barra = np.array(Xr1_barras).mean()
  desvio_Xr1_barra = np.std(np.array(Xr1_barras))
  media_Xr2_barra = np.array(Xr2_barras).mean()
  desvio_Xr2_barra = np.std(np.array(Xr2_barras))
  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_ρ2_s = np.array(ρ2_s).mean()
  desvio_ρ2_s = np.std(np.array(ρ2_s))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))


  intconf_max = (media_Xr1_barra + 1.96*(desvio_Xr1_barra/math.sqrt(len(Xr1_barras)))) * ((media_ρ1_s +  1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)))) / (media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))))  + (media_Xr2_barra + 1.96*(desvio_Xr2_barra/math.sqrt(len(Xr2_barras)))) *( (media_ρ2_s + 1.96*(desvio_ρ2_s/math.sqrt(len(ρ2_s)))) /(media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))) )
  result = (media_Xr1_barra *  (media_ρ1_s/ media_ρ_s) + media_Xr2_barra *  (media_ρ2_s/media_ρ_s))
  intconf = intconf_max - result
  print ("(p1/p) E[Xr1] + (p2/p) E[Xr2] = ",  result, " +- ", intconf  )

item6()

E[Xr] = (ρ1/ρ)E[Xr1] + (ρ2/ρ)E[Xr2]
E[Xr] =  1.50222982014645  +-  0.004407165149585905
(p1/p) E[Xr1] + (p2/p) E[Xr2] =  1.5009626823612248  +-  0.006529172636894565


#### Item 7

In [0]:
def item7():
  print("E[G] = E[X]/(1 − ρ) ")

  media_G_barra = np.array(G_barras).mean()
  desvio_G_barra = np.std(np.array(G_barras))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))
  media_X_barra = np.array(X_barras).mean()
  desvio_X_barra = np.std(np.array(X_barras))

  intconf_max = ( (media_X_barra + 1.96*(desvio_X_barra/math.sqrt(len(X_barras)) ))) /(1 - ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) )) )

  result = (  media_X_barra) /(1 - media_ρ_s ) 
  intconf = intconf_max - result

  print("E[G] = ", media_G_barra, " +- ", 1.96*(desvio_G_barra/math.sqrt(len(G_barras)) ))
  print ("E[X]/(1 − ρ) = ",  result, " +- ", intconf  )


item7()

E[G] = E[X]/(1 − ρ) 
E[G] =  6.637338027007191  +-  0.059598427492333524
E[X]/(1 − ρ) =  6.613229703828735  +-  0.05824063174261074


#### Item 8

In [0]:
def item8():
  print("E[U] = ρ1E[W1]+ρ2E[W2]+ρE[Xr] = λ1E[X1]E[W1]+λ2E[X2]E[W2]+ρE[Xr] \n")

  # print("E[X] = ", media_X_barra, " +- ", 1.96*(desvio_X_barra/math.sqrt(len(X_barras)) ))
  media_W1_barra = np.array(W1_barras).mean()
  desvio_W1_barra = np.std(np.array(W1_barras))
  media_W2_barra = np.array(W2_barras).mean()
  desvio_W2_barra = np.std(np.array(W2_barras))
  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_ρ2_s = np.array(ρ2_s).mean()
  desvio_ρ2_s = np.std(np.array(ρ2_s))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))
  media_Xr_barra = np.array(Xr_barras).mean()
  desvio_Xr_barra = np.std(np.array(Xr_barras))

  intconf_p1 = 1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)))
  intconf_p2 = 1.96*(desvio_ρ2_s/math.sqrt(len(ρ2_s)))
  intconf_p = 1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))
  intconf_w1 = 1.96*(desvio_W1_barra/math.sqrt(len(W1_barras)))
  intconf_w2 = 1.96*(desvio_W2_barra/math.sqrt(len(W2_barras)))
  intconf_xr = 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)))



  media_U_barra = np.array(U_barras).mean()
  desvio_U_barra = np.std(np.array(U_barras))

  intconf_max1 =  ((media_ρ1_s + intconf_p1)*   (media_W1_barra + intconf_w1) + (media_ρ2_s + intconf_p2) *   (media_W2_barra + intconf_w2) + (media_ρ_s + intconf_p) * (media_Xr_barra + intconf_xr))
  result1 = (media_ρ1_s *   media_W1_barra + media_ρ2_s *   media_W2_barra + media_ρ_s * media_Xr_barra)
  intconf1 = intconf_max1 - result1
  print("E[U] = ", media_U_barra, " +- ", 1.96*(desvio_U_barra/math.sqrt(len(U_barras)) ))
  print()
  print ("ρ1E[W1]+ρ2E[W2]+ρE[Xr] = ",  result1, " +- ", intconf1  )
  print()
  intconf_max2 =  l1 * (1/u1) * (media_W1_barra + intconf_w1) +  l2 * (1/u2) * (media_W2_barra + intconf_w2) + (media_ρ_s + intconf_p) * (media_Xr_barra + intconf_xr)
  result2 = l1 * (1/u1) * media_W1_barra +  l2 * (1/u2) * media_W2_barra + media_ρ_s * media_Xr_barra
  intconf2 = intconf_max2 - result2

  print ("λ1E[X1]E[W1]+λ2E[X2]E[W2]+ρE[Xr] = ",  result2, " +- ", intconf2  )

item8()

E[U] = ρ1E[W1]+ρ2E[W2]+ρE[Xr] = λ1E[X1]E[W1]+λ2E[X2]E[W2]+ρE[Xr] 

E[U] =  6.019475061097268  +-  0.08311620190259747

ρ1E[W1]+ρ2E[W2]+ρE[Xr] =  6.006639240781522  +-  0.09300317697874849

λ1E[X1]E[W1]+λ2E[X2]E[W2]+ρE[Xr] =  6.005704962930188  +-  0.0786679029816666


#### Item 9

In [0]:
def item9():
  print("ρ1E[W1]+ρ2E[W2]+ρE[Xr] = ρ1/ρ E[W1]+ ρ2/ρ E[W2] = ρE[Xr] / (1 − ρ) => ρ1E[W1]+ρ2E[W2] = ρ^2 E[Xr]/ ( 1 − ρ)\n")

  # print("E[X] = ", media_X_barra, " +- ", 1.96*(desvio_X_barra/math.sqrt(len(X_barras)) ))
  media_W1_barra = np.array(W1_barras).mean()
  desvio_W1_barra = np.std(np.array(W1_barras))
  media_W2_barra = np.array(W2_barras).mean()
  desvio_W2_barra = np.std(np.array(W2_barras))
  media_ρ1_s = np.array(ρ1_s).mean()
  desvio_ρ1_s = np.std(np.array(ρ1_s))
  media_ρ2_s = np.array(ρ2_s).mean()
  desvio_ρ2_s = np.std(np.array(ρ2_s))
  media_ρ_s = np.array(ρ_s).mean()
  desvio_ρ_s = np.std(np.array(ρ_s))
  media_Xr_barra = np.array(Xr_barras).mean()
  desvio_Xr_barra = np.std(np.array(Xr_barras))

  intconf_p1 = 1.96*(desvio_ρ1_s/math.sqrt(len(ρ1_s)))
  intconf_p2 = 1.96*(desvio_ρ2_s/math.sqrt(len(ρ2_s)))
  intconf_p = 1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)))
  intconf_w1 = 1.96*(desvio_W1_barra/math.sqrt(len(W1_barras)))
  intconf_w2 = 1.96*(desvio_W2_barra/math.sqrt(len(W2_barras)))
  intconf_xr = 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)))



  media_U_barra = np.array(U_barras).mean()
  desvio_U_barra = np.std(np.array(U_barras))

  intconf_max1 =  ((media_ρ1_s + intconf_p1)*   (media_W1_barra + intconf_w1) + (media_ρ2_s + intconf_p2) *   (media_W2_barra + intconf_w2) + (media_ρ_s + intconf_p) * (media_Xr_barra + intconf_xr))
  result1 = (media_ρ1_s *   media_W1_barra + media_ρ2_s *   media_W2_barra + media_ρ_s * media_Xr_barra)
  intconf1 = intconf_max1 - result1
  print ("ρ1E[W1]+ρ2E[W2]+ρE[Xr] = ",  result1, " +- ", intconf1  )
  print()


  intconf_max2 =  ((media_ρ1_s + intconf_p1) / (media_ρ_s + intconf_p)) * (media_W1_barra + intconf_w1) + ((media_ρ2_s + intconf_p2) / (media_ρ_s + intconf_p)) * (media_W2_barra + intconf_w2)
  result2 = (media_ρ1_s / media_ρ_s) * media_W1_barra + (media_ρ2_s / media_ρ_s) * media_W2_barra
  intconf2 = intconf_max2 - result2

  print ("ρ1/ρ E[W1]+ ρ2/ρ E[W2] = ",  result2, " +- ", intconf2  )
  print()

  intconf_max3 = (( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) ))*  (media_Xr_barra + 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)) ))) /(1 - ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) )) )

  result3 = (media_ρ_s *  media_Xr_barra) /(1 - media_ρ_s ) 
  intconf3 = intconf_max3 - result3


  print ("ρE[Xr] / (1 − ρ) = ",  result3, " +- ", intconf3  )
  print()


  intconf_max4 =  ((media_ρ1_s + intconf_p1)*   (media_W1_barra + intconf_w1) + (media_ρ2_s + intconf_p2) *   (media_W2_barra + intconf_w2) )
  result4 = (media_ρ1_s *   media_W1_barra + media_ρ2_s *   media_W2_barra )
  intconf4 = intconf_max4 - result4
  print ("ρ1E[W1]+ρ2E[W2] = ",  result4, " +- ", intconf4  )
  print()


  intconf_max5 = (( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) ))* ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) ))* (media_Xr_barra + 1.96*(desvio_Xr_barra/math.sqrt(len(Xr_barras)) ))) /(1 - ( media_ρ_s +  1.96*(desvio_ρ_s/math.sqrt(len(ρ_s)) )) )

  result5 = (media_ρ_s * media_ρ_s *  media_Xr_barra) /(1 - media_ρ_s ) 
  intconf5 = intconf_max5 - result5


  print ("ρ^2 E[Xr] / (1 − ρ) = ",  result5, " +- ", intconf5  )
  print()




item9()

ρ1E[W1]+ρ2E[W2]+ρE[Xr] = ρ1/ρ E[W1]+ ρ2/ρ E[W2] = ρE[Xr] / (1 − ρ) => ρ1E[W1]+ρ2E[W2] = ρ^2 E[Xr]/ ( 1 − ρ)

ρ1E[W1]+ρ2E[W2]+ρE[Xr] =  6.006639240781522  +-  0.09300317697874849

ρ1/ρ E[W1]+ ρ2/ρ E[W2] =  6.009414651194662  +-  0.09733545577291913

ρE[Xr] / (1 − ρ) =  5.995562276556859  +-  0.07543966879652952

ρ1E[W1]+ρ2E[W2] =  4.8053906166233515  +-  0.08717448425957652

ρ^2 E[Xr] / (1 − ρ) =  4.794313652398689  +-  0.06961097607735756



####Item 10

In [0]:
def simulaBasicoFCFS(l, u, rodadas):
  N = 0
  N_list = []
  Serverd_list = []
  inServer = []
  tempo = 0
  rodada_atual = 0

  while ( rodada_atual < rodadas):
    if (N == 0):
      chegada = random.expovariate(l)
      tempo += chegada
      inServer.append(tempo)
      N += 1
      N_list.append(N)
    else:
      temp = random.expovariate(l)
      temp2 = random.expovariate(u)
      if (temp < temp2):
        tempo += temp
        inServer.append(tempo)
        N += 1
        N_list.append(N)
      else:
        tempo += temp2
        Serverd_list.append(tempo - inServer[0])
        inServer = inServer[1:]
        N -= 1
        N_list.append(N)
    rodada_atual += 1 
  return N_list,Serverd_list 


def simulaBasicoLCFS(l, u, rodadas):
  N = 0
  N_list = []
  Serverd_list = []
  inServer = []
  tempo = 0
  rodada_atual = 0
  index_served = -1

  while ( rodada_atual < rodadas):
    if (N == 0):
      chegada = random.expovariate(l)
      tempo += chegada
      inServer.append(tempo)
      N += 1
      N_list.append(N)
      index_served = 0
    else:
      temp = random.expovariate(l)
      temp2 = random.expovariate(u)
      if (temp < temp2):
        tempo += temp
        inServer.append(tempo)
        N += 1
        N_list.append(N)
      else:
        tempo += temp2
        Serverd_list.append(tempo - inServer[index_served])
        del inServer[index_served]
        N -= 1
        index_served = len(inServer ) -1
        N_list.append(N)
    rodada_atual += 1 
  return N_list,Serverd_list 






In [42]:
lcfs = []
fcfs = []
for i in range(1000):
  N_fcfs, servered_fcfs = simulaBasicoFCFS(0.4, 1, 1000)
  N_lcfs, servered_lcfs = simulaBasicoLCFS(0.4, 1, 1000)

  fcfs.append(np.array(N_fcfs).mean())
  lcfs.append(np.array(N_lcfs).mean())

print("E[Nfcfs_1_classe]",np.array(fcfs).mean(), '+-', 1.96*(np.array(fcfs).mean()/math.sqrt(len(fcfs)) ))
print("E[Nlcfs] = ",np.array(lcfs).mean(), '+-', 1.96*(np.array(lcfs).mean()/math.sqrt(len(lcfs)) ))
print("E[Nfcfs_2_classes]",np.array(N_barras).mean(), '+-', 1.96*(np.array(N_barras).mean()/math.sqrt(len(N_barras)) ))


E[Nfcfs_1_classe] 1.1624720000000002 +- 0.07205076102895663
E[Nlcfs] =  1.159182 +- 0.07184684471631832
E[Nfcfs_2_classes] 4.05355440406667 +- 0.2512419049106407


#### Item 11

In [54]:
lcfs = []
fcfs = []
for i in range(1000):
  N_fcfs, servered_fcfs = simulaBasicoFCFS(0.4, 1, 1000)
  N_lcfs, servered_lcfs = simulaBasicoLCFS(0.4, 1, 1000)

  fcfs.append(np.array(servered_fcfs).mean())
  lcfs.append(np.array(servered_lcfs).mean())

print("E[Tfcfs_1_classe]",np.array(fcfs).mean(), '+-', 1.96*(np.array(fcfs).mean()/math.sqrt(len(fcfs)) ))
print("E[Tlcfs] = ",np.array(lcfs).mean(), '+-', 1.96*(np.array(lcfs).mean()/math.sqrt(len(lcfs)) ))
media_T = np.array(T).mean()
desvio_T = np.std(np.array(T))
print("E[Tfcfs_2_classes] = ", media_T ," +- ", 1.96*(desvio_T/math.sqrt(len(T)) ))

E[Tfcfs_1_classe] 1.6629573531930042 +- 0.10307116460117349
E[Tlcfs] =  1.6634715909484916 +- 0.10310303738747088
E[Tfcfs_2_classes] =  5.872503068765522  +-  0.06102704549636839


2.119830575869671
1.9480899397120053
